In [11]:
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (24531) start...
I (24531) just created a child process (24690).
I am child process (24690) and my parent is 24531.


In [16]:
from multiprocessing import Process
import os
import time

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()     # join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。
    print('Child process end.')

Parent process 24531.
Child process will start.
Run child process test (25067)...
Child process end.


In [58]:
from multiprocessing import Process, Pool
import os
import time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f sec.' % (name, (end - start)))

if __name__ == "__main__":
    print('parent process %s.' % os.getpid())
    init_time= time.time()
    p = Pool(4)  # pool的大小为4， 即在进程池中同时最多执行4个进程，只有等之前的线程结束后才能进入新的进程
#     init_time2= time.time()
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
#         time.sleep(0.1)
    print('Waiting for all subprocesses done...{:.2f} sec'.format(time.time() - init_time))
    p.close()
    p.join()    # 堵塞主进程，等待所有的子进程执行完毕
    print('All subProcess done.')
    print('All run {:.2f} sec'.format(time.time() - init_time))

parent process 24531.
Run task 0 (27858)...
Run task 1 (27857)...
Run task 3 (27860)...
Run task 2 (27859)...
Waiting for all subprocesses done...0.06 sec
Task 0 runs 1.10 sec.
Run task 4 (27858)...
Task 3 runs 1.85 sec.
Task 2 runs 2.48 sec.
Task 4 runs 1.42 sec.
Task 1 runs 2.82 sec.
All subProcess done.
All run 2.94 sec


In [60]:
import subprocess          # 该模块可以方便的启动一个子进程

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


### 进程间通信  
Process之间肯定是需要通信的，操作系统提供了很多机制来实现进程间的通信。Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。  
我们以Queue为例，在父进程中创建两个子进程，一个往Queue里写数据，一个从Queue里读数据：

In [119]:
import os, time, random
from multiprocessing import Process, Queue
# from queue import Queue

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())
    time.sleep(5)     # 此处延时为了等待read进程的结束

# 读数据进程执行的代码:
def read(q):
    time.sleep(5)     # 此处延时为了等待q 读取到所有的元素
    print('Process to read: %s' % os.getpid())
    while True:    # 一直在运行，等待q获取到数据
        if (not q.empty()):
            print('the queue\'s size: ', q.qsize())
            value = q.get()
            print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,), name='write')
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()

    # pr.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

Process to write: 16804
Put A to queue...
Put B to queue...
Put C to queue...
Process to read: 16805
the queue's size:  3
Get A from queue.
the queue's size:  2
Get B from queue.
the queue's size:  1
Get C from queue.


In [117]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

Process to write: 16649
Process to read: 16650
Put A to queue...
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.
